In [28]:
#from tensorflow.examples.tutorials.mnist import input_data

import tensorflow as tf
import numpy as np
#import matplotlib.pyplot as plt
#from IPython.display import set_matplotlib_formats
#from PIL import Image
#set_matplotlib_formats('pdf', 'svg')
#%matplotlib inline
import cv2
import os
import glob
from tensorflow.contrib.layers import fully_connected




from layers import conv_layer, max_pool_2x2, full_layer

DATA_DIR = '/tmp/data'
MINIBATCH_SIZE = 50
STEPS = 5000


test_path  = "/Users/gerritbecker/Documents/Python/CNN_mnist_test/TF_Images/test/"
train_path = "/Users/gerritbecker/Documents/Python/CNN_mnist_test/TF_Images/train/"


def next_batch(path, size, GRBtoGray=False, flatten=False):
    labels = []
    with open(path + "trainlabels.txt", "r") as r_txtfile:
        labels = r_txtfile.read().splitlines()
        labels = np.array(labels)  # noch als strings

    data_path = os.path.join(path, '*g')  # ???
    files = glob.glob(data_path)
    num_imag = len(files)
    Data = np.arange(0, num_imag)
    batchindices = np.random.choice(Data, size=size,
                                    replace=False)  # Zufallsstichprobe; reihenfolge beliebig; ohne zurückl.
    labels = labels[batchindices]
    labels = [(int(i) - 1) for i in labels]
    labels = np.array(labels, dtype = np.uint8)

    img_list = []
    for i, f1 in enumerate(files):
        if i in batchindices:
            if GRBtoGray == True:
                img = cv2.imread(f1, flags=0)
            else:
                img = cv2.imread(f1, flags=1)

            if flatten == True:
                img_list.append(img.flatten())
            else:
                img_list.append(img)

    img_list = np.array(img_list)
    #  img_list = np.reshape(img_list,(size,50*50,))

    return img_list, labels




def test_images(path, size, GRBtoGray=False, flatten=False):
    labels = []
    with open(path + "testlabels.txt", "r") as r_txtfile:
        labels = r_txtfile.read().splitlines()
        labels = np.array(labels)  # noch als strings

    data_path = os.path.join(path, '*g')  # ???
    files = glob.glob(data_path)
    num_imag = len(files)
    Data = np.arange(0, num_imag)
    batchindices = np.random.choice(Data, size=size,
                                    replace=False)  # Zufallsstichprobe; reihenfolge beliebig; ohne zurückl.
    labels = labels[batchindices]
    labels = [(int(i) - 1) for i in labels]
    labels = np.array(labels, dtype= np.uint8)

    img_list = []
    for i, f1 in enumerate(files):
        if i in batchindices:
            if GRBtoGray == True:
                img = cv2.imread(f1, flags=0)
            else:
                img = cv2.imread(f1, flags=1)

            if flatten == True:
                img_list.append(img.flatten())
            else:
                img_list.append(img)

    img_list = np.array(img_list)

    return img_list, labels




#mnist = input_data.read_data_sets(DATA_DIR, one_hot=True)

x = tf.placeholder(tf.float32, shape=[None, 28, 28, 1])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

#x_image = tf.reshape(x, [-1, 28, 28, 1])
conv1 = conv_layer(x, shape=[5, 5, 1, 32])
conv1_pool = max_pool_2x2(conv1)

conv2 = conv_layer(conv1_pool, shape=[5, 5, 32, 64])
conv2_pool = max_pool_2x2(conv2)

conv2_flat = tf.reshape(conv2_pool, [-1, 7*7*64])
full_1 = tf.nn.relu(full_layer(conv2_flat, 1024))

keep_prob = tf.placeholder(tf.float32)
full1_drop = tf.nn.dropout(full_1, keep_prob=keep_prob)

y_conv = full_layer(full1_drop, 10)

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_conv, labels=y_))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for i in range(STEPS):
        batch = next_batch(path=train_path, size=MINIBATCH_SIZE, GRBtoGray=True, flatten=False)
        #batch[0] = np.expand_dims(batch[0], 3)
        #y_batch.astype(dtype= np.uint8)
        #batch = .train.next_batch(MINIBATCH_SIZE)

        if i % 100 == 0:
            train_accuracy = sess.run(accuracy, feed_dict={x: batch[0], y_: batch[1],
                                                           keep_prob: 1.0})
            #train_accuracy = sess.run(accuracy, feed_dict={x: X_batch, y_: y_batch,
             #                                              keep_prob: 1.0})
            print("step {}, training accuracy {}".format(i, train_accuracy))

        sess.run(train_step, feed_dict={x: X_batch, y_: y_batch, keep_prob: 0.5})

    #X = mnist.test.images.reshape(10, 1000, 784)
    #Y = mnist.test.labels.reshape(10, 1000, 10)
    #test_accuracy = np.mean(
    #    [sess.run(accuracy, feed_dict={x: X[i], y_: Y[i], keep_prob: 1.0}) for i in range(10)])

#print("test accuracy: {}".format(test_accuracy))


ValueError: Cannot feed value of shape (50, 28, 28) for Tensor 'Placeholder_30:0', which has shape '(?, 28, 28, 1)'

In [23]:
y_batch

array([0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0,
       1, 0, 1, 0], dtype=uint8)

In [6]:
np.expand_dims(X_batch, 3).shape

(50, 28, 28, 1, 1)

In [20]:
        X_batch, y_batch = next_batch(path=train_path, size=1500, GRBtoGray=True, flatten=False)



In [11]:
a= 1,2

In [12]:
a

(1, 2)

In [14]:
a[0]

1